<a href="https://colab.research.google.com/github/hucarlos08/GEO-ML/blob/main/Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes
The following code is based on the scikit-learn documentation titled 'Common pitfalls in the interpretation of coefficients of linear models'. You can find the original documentation at:
<br>
https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html.
<br>
The code provided in this example includes explanations added to the functions and utilizes Plotly instead of Seaborn for generating the graphs. The purpose of this code is to demonstrate common pitfalls that can arise when interpreting the coefficients of linear models. By exploring the code and its visualizations, you can gain insights into the potential challenges and considerations involved in interpreting the coefficients of linear models.

# Objective
Let's explore the Sklearn pipeline for linear regression in more detail. The objective is to leverage a publicly available dataset in order to demonstrate the pipeline's functionality.

The Sklearn pipeline for linear regressions provides a powerful toolset for data analysis and modeling. By employing a sequential workflow, it allows for streamlined data preprocessing, feature engineering, and model training.

To illustrate the pipeline, we will utilize a publicly accessible dataset, such as the OpenMP dataset. OpenMP datasets are valuable resources for various machine learning tasks, offering diverse and well-curated data. By utilizing such datasets, we can demonstrate the effectiveness of the Sklearn pipeline in a real-world context.

In [34]:
# Libraries
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

## Load the data

In [35]:
""" the provided code loads a specific dataset from OpenML (identified by data_id) using Scikit-learn's fetch_openml 
function. The dataset is returned as a pandas DataFrame (as_frame=True) and is then printed to provide an overview 
of its contents. 
"""
from sklearn.datasets import fetch_openml

survey = fetch_openml(data_id=43308, as_frame=True, parser="pandas")

""" survey.data.info(), you will see an output that provides information about the structure and properties of the 
dataset, giving you a quick overview of the available columns and their data types. This information can be helpful 
for understanding the dataset's characteristics and planning subsequent data processing or analysis tasks.
"""
survey.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Species  159 non-null    object 
 1   Length1  159 non-null    float64
 2   Length2  159 non-null    float64
 3   Length3  159 non-null    float64
 4   Height   159 non-null    float64
 5   Width    159 non-null    float64
dtypes: float64(5), object(1)
memory usage: 7.6+ KB


## Features and target variable

### Features

In [36]:
""" This line extracts a subset of columns from the data attribute of the survey object. The survey.feature_names 
contains a list of column names representing the features in the dataset. By using this list to index the data 
attribute, we create a new DataFrame X that only includes the specified columns.
"""

X = survey.data[survey.feature_names]

"""  The describe() method provides descriptive statistics for each column in the DataFrame. By specifying include="all", 
it includes both numeric and categorical columns in the summary.
"""

X.describe(include="all")

,Species,Length1,Length2,Length3,Height,Width
count,159,159.000000,159.000000,159.000000,159.000000,159.000000
unique,7,NaN,NaN,NaN,NaN,NaN
top,Perch,NaN,NaN,NaN,NaN,NaN
freq,56,NaN,NaN,NaN,NaN,NaN
mean,NaN,26.247170,28.415723,31.227044,8.970994,4.417486
std,NaN,9.996441,10.716328,11.610246,4.286208,1.685804
min,NaN,7.500000,8.400000,8.800000,1.728400,1.047600
25%,NaN,19.050000,21.000000,23.150000,5.944800,3.385650
50%,NaN,25.200000,27.300000,29.400000,7.786000,4.248500
75%,NaN,32.700000,35.500000,39.650000,12.365900,5.584500


### Target variable

In [37]:
""" This line displays the first few rows of the target variable in the survey object. The head() method is commonly 
used to inspect the top rows of a DataFrame or Series. By calling head() on the target attribute, you can quickly 
examine the initial values of the target variable.
"""
print(survey.target.head())

""" This line extracts the target variable from the survey object and assigns it to the variable y. The target 
attribute typically contains the outcome or dependent variable of the dataset. In this case, the values.ravel() method 
is used to flatten the target values into a 1-dimensional array. This is often necessary to ensure compatibility 
with certain Scikit-learn functions that expect a 1-dimensional target array.
"""
y = survey.target.values.ravel()

0    242.0
1    290.0
2    340.0
3    363.0
4    430.0
Name: Weight, dtype: float64


## Split in train and test

In [56]:
""" The dataset is divided randomly into training and testing sets based on the specified random_state. The training set 
is typically used to train the machine learning model, while the testing set is used for evaluating the model's 
performance on unseen data. 
"""
from sklearn.model_selection import train_test_split

""" random_state=42: This parameter sets the random seed to ensure reproducibility of the same train-test split. 
Setting random_state to a specific value (here, 42) ensures that the same split is obtained every time the code is run.
"""

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Some insights

In [57]:
import plotly.express as px

""" This line creates a copy of the training feature variables (X_train) and assigns it to the train_dataset variable. 
It is often a good practice to work with a copy of the data to avoid unintended modifications to the original dataset.
"""
train_dataset = X_train.copy()

""" This line inserts a new column called "WAGE" at the beginning of the train_dataset DataFrame. The values of this 
column are taken from the target variable (y_train). This step combines the feature variables and target variable 
together in a single DataFrame, making it easier to visualize their relationships.
"""
train_dataset.insert(0, "Weight", y_train)

fig = px.scatter_matrix(train_dataset, dimensions=train_dataset.columns, opacity=0.7, color="Species", 
                        symbol='Species', width=1500, height=1200)
fig.update_traces(diagonal_visible=False)
fig.update_layout(title="Pair Plot of Training Dataset")
fig.show()

## Categorical variables

In [58]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

""" By creating this column transformer, you can apply the specified transformations to the categorical and numerical 
columns in your dataset. It allows for flexible preprocessing and handling of different column types within 
a unified pipeline.
"""

categorical_columns = ["Species"]
numerical_columns = ["Length1", "Length2", "Length3", "Height", "Width"]

""" 
1. (OneHotEncoder(drop="if_binary"), categorical_columns): This specifies a transformation step for the categorical 
columns. The OneHotEncoder is used to perform one-hot encoding on these columns. 
2. The drop="if_binary" parameter indicates that if a categorical column has only two unique values, one of them will be 
dropped to avoid multicollinearity.
3. remainder="passthrough": This parameter indicates that any remaining columns that are not explicitly specified should
be passed through without any transformations.
4. verbose_feature_names_out=False: This parameter is set to False to prevent the transformer from prepending its name 
to the feature names, resulting in more concise feature names.
"""
preprocessor = make_column_transformer(
    (OneHotEncoder(drop="if_binary"), categorical_columns),
    remainder="passthrough",
    verbose_feature_names_out=False,  # avoid to prepend the preprocessor names
)

## The pipeline

The **make_pipeline** function is called to create the machine learning pipeline, model. It takes a series of steps that are applied in sequential order.

1. **preprocessor**: This is the column transformer created earlier, which applies the necessary preprocessing steps to the data.
2. **TransformedTargetRegressor**: This is a meta-estimator that wraps the Ridge regressor with a transformation on the target variable. It allows for applying a transformation to the target variable during training and inverse transformation during prediction.
3. **regressor=Ridge(alpha=1e-10)**: This specifies the Ridge regressor as the underlying estimator for the meta-estimator. The alpha parameter is set to 1e-10, which represents a very small regularization strength.

*By creating this pipeline, you can seamlessly apply the preprocessing steps using the preprocessor and train a Ridge regression model with transformed target variables using the TransformedTargetRegressor. The pipeline allows for a streamlined and reproducible machine learning workflow.*

In [59]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.compose import TransformedTargetRegressor

model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=Ridge(alpha=1e-10), func=None, inverse_func=None
    ),
)

## Fit the model

In [60]:
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['Species'])],
                                   verbose_feature_names_out=False)),
                ('transformedtargetregressor',
                 TransformedTargetRegressor(regressor=Ridge(alpha=1e-10)))])

## The performance

This code provides an evaluation of the model's performance by calculating the MedAE on both the training and testing sets and displaying the results. The MedAE is a measure of the average absolute difference between the true and predicted target values, with a lower value indicating better predictive accuracy.

In [134]:
from sklearn.metrics import median_absolute_error
from sklearn.metrics import PredictionErrorDisplay

""" The median_absolute_error function takes two arguments: the true target values and the predicted target values for 
each dataset (Train and Test).
"""
mae_train = median_absolute_error(y_train, model.predict(X_train))
mae_test  = median_absolute_error(y_test,  model.predict(X_test))


scores = {
    "MedAE on training set": f"{mae_train:.2f}",
    "MedAE on testing set": f"{mae_test:.2f}",
}

print(scores)

{'MedAE on training set': '54.08', 'MedAE on testing set': '58.65'}


In [65]:
import pandas as pd

# Creating a DataFrame for the training set with a column 'Train' for differentiation
train_df = pd.DataFrame(X_train, columns=X.columns)
train_df['Target'] = y_train
train_df['Data'] = 'Train'
train_df['Prediction'] = model.predict(X_train)

# Creating a DataFrame for the testing set with a column 'Test' for differentiation
test_df = pd.DataFrame(X_test, columns=X.columns)
test_df['Target'] = y_test
test_df['Data'] = 'Test'
test_df['Prediction'] = model.predict(X_test)


# Concatenating the training and testing DataFrames
combined_df = pd.concat([train_df, test_df], ignore_index=True)

fig = px.scatter(
    combined_df, x='Target', y='Prediction',
    marginal_x='box', marginal_y='box',
    color='Data', trendline='ols'
)
fig.update_traces(histnorm='probability', selector={'type':'histogram'})
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=y.min(), y0=y.min(),
    x1=y.max(), y1=y.max()
)

fig.show()

By visualizing the target variable and the predicted values in a scatter plot, this code helps assess the performance of the machine learning model. It allows users to evaluate how well the predictions align with the actual values and identify any patterns or discrepancies. The inclusion of marginal box plots and a trendline provides additional insights into the distribution and trend of the data. Overall, this visualization aids in understanding the model's performance and identifying potential areas of improvement.

## Coefficients: scale matters

In [133]:
""" This line retrieves the names of the features from the model object. Since the model is a pipeline, model[:-1] 
refers to all the steps in the pipeline except for the last step. The get_feature_names_out() method is then called on 
the extracted pipeline to obtain the names of the transformed features.
"""

feature_names = model[:-1].get_feature_names_out()

""" This code creates a DataFrame called coefs to store the coefficients of the features. It uses the regressor_ 
attribute of the last step in the pipeline (model[-1]) to access the underlying linear regression model. 
The coef_ attribute retrieves the coefficients of the linear regression model. These coefficients are then passed as 
data to the DataFrame along with the column name "Coefficients" and the index set to the feature names obtained 
earlier.
"""

coefs = pd.DataFrame(
    model[-1].regressor_.coef_,
    columns=["Coefficients"],
    index=feature_names,
)

print(coefs.head(10))

colors = ['Positive' if c > 0 else 'Negative' for c in coefs['Coefficients']]

fig = px.bar(
    x=coefs.index, y=coefs['Coefficients'], color=colors,
    color_discrete_sequence=['red', 'blue'],
    labels=dict(x='Feature', y='Linear coefficient'),
    title='Weight of each feature for predicting'
)
fig.show()

                   Coefficients
Species_Bream        -40.050592
Species_Parkki       101.206854
Species_Perch         21.166574
Species_Pike        -371.318421
Species_Roach         -3.072985
Species_Smelt        290.096116
Species_Whitefish      2.008720
Length1              -69.420081
Length2               72.442845
Length3               32.642834


## Coefficient variability 

 This code performs cross-validation, extracts the trained models' coefficients from each fold, and computes a DataFrame (coefs) containing the standardized coefficients for each feature across the cross-validation folds. This analysis can provide insights into the stability and significance of the feature coefficients in the model.

In [131]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold

""" This line initializes the cv object as a RepeatedKFold instance. It specifies the number of splits (n_splits=5) 
and the number of repeats (n_repeats=5) for the cross-validation. The random_state parameter sets the random seed 
for reproducibility. cross_validate is used to perform cross-validation, while RepeatedKFold is a strategy for splitting 
the data into train and test sets during cross-validation.
"""
cv = RepeatedKFold(n_splits=5, n_repeats=5, random_state=0)

""" This code performs cross-validation using the cross_validate function. It takes the machine learning model, 
feature data X, and target variable y as inputs. The cv parameter specifies the cross-validation strategy (in this case,
the cv object we defined earlier). Setting return_estimator=True ensures that the trained estimators from each fold 
are returned. The n_jobs parameter sets the number of parallel jobs to run during cross-validation.
"""

cv_model = cross_validate(
    model,
    X,
    y,
    cv=cv,
    return_estimator=True,
    n_jobs=2,
)

""" This code creates a DataFrame called coefs to store the coefficients of the features from each cross-validated model. 
It loops over the estimators (est) and their corresponding train indices (train_idx) obtained from cv_model["estimator"] 
and cv.split(X, y), respectively. For each estimator, it multiplies the coefficients (est[-1].regressor_.coef_) 
with the standard deviation (est[:-1].transform(X.iloc[train_idx]).std(axis=0)) of the transformed features 
(est[:-1].transform(X.iloc[train_idx]))
"""
coefs = pd.DataFrame(
    [
        est[-1].regressor_.coef_ * est[:-1].transform(X.iloc[train_idx]).std(axis=0)
        for est, (train_idx, _) in zip(cv_model["estimator"], cv.split(X, y))
    ],
    columns=feature_names,
)

# Plot
""" The melted DataFrame has a separate column named 'Column', which represents the original column names 
('Column1', 'Column2', 'Column3'). The values of these columns are placed in a new column named 'Value'.
We then pass the 'Column' column as the x parameter, the 'Value' column as the y parameter, and the 'Column' column as 
the color parameter in the px.box() function. This assigns different colors to each boxplot based on the unique values 
in the 'Column' column.
"""
df_melted = coefs.melt(var_name='Column', value_name='Value')

fig = px.box(df_melted,  y='Column', x='Value', color='Column', width=1000, height=800)
fig.show()

In [130]:
import random

""" This line defines a list of column names (column_to_drop) that need to be dropped from the feature data.
"""
column_to_drop = ["Length2",'Length3']

""" This code performs cross-validation on the machine learning model. It drops the columns specified in column_to_drop 
from the feature data (X) using X.drop(columns=column_to_drop). The remaining feature data is used along with the target 
variable (y) to perform cross-validation.
"""
cv_model = cross_validate(
    model,
    X.drop(columns=column_to_drop),
    y,
    cv=cv,
    return_estimator=True,
    n_jobs=2,
)

""" This code creates a list (features_name_list) of feature names by converting feature_names (obtained previously) to 
a Python list. Then, it removes the names of the dropped columns (column_to_drop) from the list.
"""
features_name_list = feature_names.tolist()
for drop_name in column_to_drop:
  features_name_list.remove(drop_name)


coefs = pd.DataFrame(
    [
        est[-1].regressor_.coef_
        * est[:-1].transform(X.drop(columns=column_to_drop).iloc[train_idx]).std(axis=0)
        for est, (train_idx, _) in zip(cv_model["estimator"], cv.split(X, y))
    ],
    columns=features_name_list,
)

# Plot
""" The melted DataFrame has a separate column named 'Column', which represents the original column names 
('Column1', 'Column2', 'Column3'). The values of these columns are placed in a new column named 'Value'.
We then pass the 'Column' column as the x parameter, the 'Value' column as the y parameter, and the 'Column' column as 
the color parameter in the px.box() function. This assigns different colors to each boxplot based on the unique values 
in the 'Column' column.
"""
df_melted = coefs.melt(var_name='Column', value_name='Value')

fig = px.box(df_melted,  y='Column', x='Value', color='Column', width=1000, height=800)
fig.show()

## Preprocessing numerical variables

In this case, it creates a preprocessor that consists of two transformers:

1. The first transformer, **OneHotEncoder**, is applied to the *categorical_columns* and specifies drop="if_binary". It encodes categorical features using a one-hot encoding scheme, which creates binary columns for each unique category. The drop parameter specifies that if a categorical feature has only two unique categories, one of the binary columns should be dropped to avoid multicollinearity.

1. The second transformer, **StandardScaler**, is applied to the *numerical_columns*. It standardizes numerical features by removing the mean and scaling to unit variance. This transformation ensures that the numerical features have similar scales, which can be beneficial for certain models and algorithms.

By combining these transformers using make_column_transformer, you can create a single preprocessor that can handle both categorical and numerical features.

In [135]:
from sklearn.preprocessing import StandardScaler

preprocessor = make_column_transformer(
    (OneHotEncoder(drop="if_binary"), categorical_columns),
    (StandardScaler(), numerical_columns),
)

In [137]:
model = make_pipeline(
    preprocessor,
    TransformedTargetRegressor(
        regressor=Ridge(alpha=1e-10), func=None, inverse_func=None
    ),
)
model.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['Species']),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  ['Length1', 'Length2',
                                                   'Length3', 'Height',
                                                   'Width'])])),
                ('transformedtargetregressor',
                 TransformedTargetRegressor(regressor=Ridge(alpha=1e-10)))])

## The performance AGAIN

In [139]:
mae_train = median_absolute_error(y_train, model.predict(X_train))
mae_test  = median_absolute_error(y_test,  model.predict(X_test))


scores = {
    "MedAE on training set": f"{mae_train:.2f}",
    "MedAE on testing set": f"{mae_test:.2f}",
}

print(scores)

{'MedAE on training set': '54.08', 'MedAE on testing set': '58.65'}


In [138]:
# Creating a DataFrame for the training set with a column 'Train' for differentiation
train_df = pd.DataFrame(X_train, columns=X.columns)
train_df['Target'] = y_train
train_df['Data'] = 'Train'
train_df['Prediction'] = model.predict(X_train)

# Creating a DataFrame for the testing set with a column 'Test' for differentiation
test_df = pd.DataFrame(X_test, columns=X.columns)
test_df['Target'] = y_test
test_df['Data'] = 'Test'
test_df['Prediction'] = model.predict(X_test)


# Concatenating the training and testing DataFrames
combined_df = pd.concat([train_df, test_df], ignore_index=True)

fig = px.scatter(
    combined_df, x='Target', y='Prediction',
    marginal_x='box', marginal_y='box',
    color='Data', trendline='ols'
)
fig.update_traces(histnorm='probability', selector={'type':'histogram'})
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=y.min(), y0=y.min(),
    x1=y.max(), y1=y.max()
)

fig.show()

## Things to do..
In the exercise, we examined the variation in features when removing those with high correlation. However, we didn't evaluate the impact on the model's performance after removing these features. Moreover, the value of the alpha parameter in the Ridge regression was very small. Let's explore the effects of increasing this parameter and its impact on the model's performance.

To accomplish this, we can create a pipeline that utilizes scikit-learn's RidgeCV function to find the optimal alpha automatically. The RidgeCV function performs ridge regression with built-in cross-validation to determine the best alpha value.

Implement the pipeline and assess the model's performance with different alpha values to identify the optimal alpha for our dataset.Hhere is a prototype

In [144]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Step 1: Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='if_binary'), categorical_columns),
        ('imputer', SimpleImputer(strategy='mean'), numerical_columns),
        ('scaler', StandardScaler(), numerical_columns)
    ])

# Step 2: Ridge regression pipeline
ridge_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('ridge', RidgeCV(alphas=[0.1, 1.0, 10.0]))  # Specify alpha values to be tested
])

# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Fit the model and make predictions
ridge_pipeline.fit(X_train, y_train)
y_pred = ridge_pipeline.predict(X_test)

# Step 5: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Best alpha value:", ridge_pipeline['ridge'].alpha_)


Mean Squared Error: 6950.271561674497
Best alpha value: 0.1
